# Проект: pretraining LLM и posttraining LLM

## Постановка задачи

- Pretraining
    - Претрейн — самый ресурсоёмкий этап обучения LLM. Чтобы полноценно обучить даже небольшую модель (менее 1B), понадобится более 10к GPU-часов на A100. Чтобы не тратить недели на обучение, но отработать ключевые приёмы, в проекте вы выполните упрощённую задачу. 
    - При полноценном претрейне модель учится обобщать знания из данных, на которых происходило обучение, чтобы потом извлекать эти знания по текстовым запросам уже после обучения. Упростим задачу — научим модель только структуре языка. 
    - Сосредоточимся на одном узком домене — **текстах произведений русской литературы** — и **обучим модель продолжать фразы из этого домена разумным текстом**. 
- Posttraining
    - Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель **Qwen2.5-0.5B**, с которой вы встречались в уроках. **Обучите её генерировать ответы на инструктивные русскоязычные вопросы**.

## Критерии успеха
- Pretraining - чтобы оценить качество, используйте промпты:
    - "Все мысли, которые имеют огромные последствия",
    - "Сила войска зависит от его духа",
    - "Мысль о том, что он принес страдания",
    - "Человек сознает себя свободным",
    - "Что бы ни случилось, я всегда буду",
    - "Любовь мешает смерти",
    - "Нет, жизнь не кончена",
    - "Всякая мысль, даже самая простая",
    - "Война не любезность, а самое гадкое дело",
    - "Чтобы жить честно"
- Posttraining - качество данных должно быть сопоставимо с таким вариантом::
    - Model Input 1:
    сколько планет в нашей солнечной системе?
        - Model Output 1:
    Согласно последним исследованиям, Солнце имеет 8 планеты. Это дает планетам, которые соответствуют разным критериям, таким как диета, местоположение, плотность, климат и другие.
    assistant
    Общее размер нашей Солнечной системы составляет примерно 9,9 миллиарда километров, а в результате на каждом из них есть планета,
    - Model Input 2:
    расскажи стих
        - Model Output 2:
    Вот сладкий сладкий вкус, который ты получаешь на себе
    Когда я говорю, что я люблю тебя
    Мое сердце, у меня есть все, что я хочу, чтобы сделать
    У меня есть все, что мне нужно, чтобы быть хорошим
    Так давай пойдем в ресторан
    assistant
    Пока мы уседим, что мы можем
    И в этот момент мы возвращаемся к при
    - Model Input 3:
    когда собирать крыжовник?
        - Model Output 3:
    Когда собираешь крыжовник, важно не беспокоиться о том, что другие могут его хвастаться. Вместо этого проверяйте свои способности и готовность. Вы можете изучить свой талант и стремления, прежде чем принимать решение, и признавать, что у вас есть свои сильные стороны, а также слабые места. Не снимайте крыжовник в обществе, и обязательно помните о
    - Model Input 4:
    Как быстро выучить новый язык?
        - Model Output 4:
    Скорость усвоения нового языка может сильно различаться в зависимости от нескольких факторов, таких как скорость, с которой вы пакетируете информацию через ее и ее способность учиться в различных условиях. Тем не менее, вот некоторые общие рекомендации: 1. Делайте сценарии. Обычно это может занять от 2 до 4 лет, чтобы понять основные концепции. Вы можете начать с

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных

- Pretrain
    - https://github.com/JoannaBy/RussianNovels/tree/master/corpus 
- Posttrain
    - русскоязычный инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) в диалоговом формате

## Инициализация

### Импорт библиотек

In [3]:
!pip install GitPython

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
import re
from git import Repo
from pathlib import Path
from torch.utils.data import Dataset
import pandas as pd
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from tokenizers.models import BPE

### Установка главных параметров

In [5]:
PRETRAIN_DATASET =  "pretrain-dataset"

# Pretraining

Скачайте данные из репозитория и упакуйте их в один датасет. Вам понадобятся все произведения из репозитория.

Проведите препроцессинг данных:
- Очистите их от дубликатов.
- Очистите от предложений с буквами не из кириллицы.
- Обработайте повторяющуюся пунктуацию и т. д.
- Разбейте на чанки поменьше, чтобы можно было добавить <bos> и <eos> токены в соответствии с обучаемой длиной контекста.

In [6]:
if not os.path.isdir(PRETRAIN_DATASET):
    Repo.clone_from("https://github.com/JoannaBy/RussianNovels.git", PRETRAIN_DATASET) 

In [7]:
class PretrainedDataset(Dataset):
    def __init__(self, text_files_dir):
        self.texts = []
        for file_path in Path(text_files_dir).glob("*.txt"):
            with file_path.open("r", encoding="utf-8") as f:
                for line in f:
                    self.texts.append(line.strip())
        self.clean_data()
        
    def clean_data(self):
        # удалим строки с недопустимыми символами
        allowed_pattern = re.compile(r"^[А-Яа-яЁё0-9.,!?;:\-()\"'\s]+$")
        self.texts = [t for t in self.texts if allowed_pattern.fullmatch(t)]

        # Заменяем повторяющиеся знаки пунктуации на один
        self.texts = [re.sub(r'([.,!?;:\-()])\1+', r'\1', t) for t in self.texts]

        #удалим дубликаты
        self.texts = pd.DataFrame({'t': self.texts}).drop_duplicates()['t'].tolist()

        #перенесём строки, если они длиннее 300 слов
        max_words = 300 # из рассчёта, что ~1.5 токена на слово, а токенов у нас в контексте 512
        result = []

        for text in self.texts:
            words = text.split()
            if len(words) > max_words:
                parts = [' '.join(words[i:i + max_words]) \
                            for i in range(0, len(words), max_words)]
                result.extend(parts)
            else:
                result.append(text)
            
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

In [8]:
pretrained_dataset = PretrainedDataset(Path(PRETRAIN_DATASET, "corpus"))

In [9]:
pretrained_dataset.__len__()

270498

In [10]:
pretrained_dataset.__getitem__(100)

{'text': '- Тяжкое, тяжкое время, что говорить, - пробормотал он, - но унывать-то'}

Создайте и обучите собственный токенизатор на полученных данных. Размер словаря выберите небольшим: при обучении только на рассмотренных текстах — около 3к токенов. В рассматриваемых данных язык намного менее разнообразен, чем в совокупных данных, поэтому крупные токенизаторы от реальных LLM могут не подойти. 
При создании токенизатора можете ориентироваться на [материал huggingface.co](https://huggingface.co/learn/llm-course/ru/chapter6/8). Рекомендуем использовать BPE.

In [ ]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)